In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from __future__ import print_function, division
from re import X

import tensorflow

from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import (
    Input,
    Dense,
    Reshape,
    Flatten,
    Dropout,
    multiply,
    GaussianNoise,
    Conv2D,
    Conv2DTranspose,
    LeakyReLU,
)
from tensorflow.keras.layers import (
    BatchNormalization,
    Activation,
    Embedding,
    ZeroPadding2D,
)
from tensorflow.keras.layers import MaxPooling2D, concatenate, Resizing, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import losses
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

import matplotlib.pyplot as plt

import numpy as np
from numpy import load

In [ ]:
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)


class BIGAN:
    def __init__(self):
        self.img_rows = 80
        self.img_cols = 80
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        print("img shape", self.img_shape)

        optimizer = tensorflow.keras.optimizers.Adam(0.0001, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=["binary_crossentropy"], optimizer=optimizer, metrics=["accuracy"]
        )

        # Build the generator
        self.generator = self.build_generator()

        # Build the encoder
        self.encoder = self.build_encoder()

        # The part of the bigan that trains the discriminator and encoder
        self.discriminator.trainable = False

        # Generate image from sampled noise
        z = Input(shape=(self.latent_dim,))
        img_ = self.generator(z)

        print(img_.shape)

        # Encode image
        img = Input(shape=self.img_shape)
        z_ = self.encoder(img)

        print(z_.shape)

        # Latent -> img is fake, and img -> latent is valid
        fake = self.discriminator([z, img_])
        valid = self.discriminator([z_, img])

        # Set up and compile the combined model
        # Trains generator to fool the discriminator
        self.bigan_generator = Model([z, img], [fake, valid])
        self.bigan_generator.compile(
            loss=["binary_crossentropy", "binary_crossentropy"], optimizer=optimizer
        )

    def build_encoder(self):

        model = Sequential()
        # normal
        model.add(Conv2D(32, (5, 5), padding="same", input_shape=self.img_shape))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.1))
        # downsample to 40x40
        model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.1))
        # downsample to 20x20
        model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.1))
        # downsample to 10x10
        model.add(Conv2D(128, (5, 5), strides=(2, 2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.1))
        # downsample to 5x5
        model.add(Conv2D(128, (5, 5), strides=(2, 2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Flatten())
        model.add(Dropout(0.1))
        # model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(Dense(self.latent_dim))
        # model.add(Dense(self.latent_dim, activation="sigmoid"))

        print("encoder")

        img = Input(shape=self.img_shape)
        z = model(img)

        return Model(img, z)

    def build_generator(self):
        model = Sequential()

        n_nodes = 256 * 5 * 5
        model.add(Dense(n_nodes, input_dim=self.latent_dim))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Reshape((5, 5, 256)))
        # upsample to 10x10
        model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        # upsample to 20x20
        model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        # upsample to 40x40
        model.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        # upsample to 80x80
        model.add(Conv2DTranspose(32, (4, 4), strides=(2, 2), padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Conv2D(self.channels, (5, 5), activation="tanh", padding="same"))

        print("generator")

        z = Input(shape=(self.latent_dim,))
        gen_img = model(z)

        return Model(z, gen_img)


    def build_discriminator(self):

        inputA = Input(shape=(self.latent_dim,))
        inputB = Input(shape=(self.img_rows, self.img_cols, self.channels))

        # normal
        y = Conv2D(32, (7, 7), padding="same", input_shape=self.img_shape)(inputB)
        y1 = LeakyReLU(alpha=0.2)(y)
        #
        y = Conv2D(64, (3, 3), strides=(2,2), padding="same")(y1)
        y = LeakyReLU(alpha=0.2)(y)
        #
        y = Conv2D(64, (3, 3), padding="same")(y)

        y1 = Conv2D(64, (3, 3), strides=(2,2), padding="same")(y1)

        y = Add()([y1, y])
        y11 = LeakyReLU(alpha=0.2)(y)
        #------------

        y = Conv2D(128, (3, 3), strides=(2,2), padding="same")(y11)
        y = LeakyReLU(alpha=0.2)(y)

        y = Conv2D(128, (3, 3), padding="same")(y)

        y2 = Conv2D(128, (3, 3), strides=(2,2), padding="same")(y11)

        y = Add()([y2, y])
        y22 = LeakyReLU(alpha=0.2)(y)
        #------------

        y = Conv2D(128, (3, 3), strides=(2,2), padding="same")(y22)
        y = LeakyReLU(alpha=0.2)(y)

        y = Conv2D(128, (3, 3), padding="same")(y)

        y3 = Conv2D(128, (3, 3), strides=(2,2), padding="same")(y22)

        y = Add()([y3, y])
        y33 = LeakyReLU(alpha=0.2)(y)
        #------------

        y = Conv2D(256, (3, 3), strides=(2,2), padding="same")(y33)
        y = LeakyReLU(alpha=0.2)(y)

        y = Conv2D(256, (3, 3), padding="same")(y)

        y4 = Conv2D(256, (3, 3), strides=(2,2), padding="same")(y33)

        y = Add()([y4, y])
        y44 = LeakyReLU(alpha=0.2)(y)
        #------------
        y = Conv2D(256, (3, 3), padding="same")(y44)
        y = LeakyReLU(alpha=0.2)(y)
        y = Flatten()(y)

        d = Dense(256)(inputA)

        combined = concatenate([d, y])

        z = Dense(128)(combined)
        z = Dense(1, activation="softmax")(z)

        return Model(inputs=[inputA, inputB], outputs=z)

    def train(self, epochs, batch_size=125, sample_interval=50):

        # Load the dataset
        X_train = load("./drive/MyDrive/img_align_celeba_80x80_50k_color.npz")
        X_train = X_train["arr_0"]

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        # idx = 0
        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Sample noise and generate img
            z = np.random.normal(size=(batch_size, self.latent_dim))
            imgs_ = self.generator.predict(z)

            #Select a random batch of images and encode
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            z_ = self.encoder.predict(imgs)

            # Train the discriminator (img -> z is valid, z -> img is fake)
            d_loss_real = self.discriminator.train_on_batch([z_, imgs], valid)
            d_loss_fake = self.discriminator.train_on_batch([z, imgs_], fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (z -> img is valid and img -> z is is invalid)
            g_loss = self.bigan_generator.train_on_batch([z, imgs], [valid, fake])

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:

                print(
                "%d [D loss: %f, acc: %.2f%%] [G loss: %f]"
                % (epoch, d_loss[0], 100 * d_loss[1], g_loss[0])
                )
                self.sample_interval(epoch)
                self.generator.save(f"./drive/MyDrive/gan_models3/gen_e_{epoch}.h5")
                self.encoder.save(f"./drive/MyDrive/gan_models3/encoder_e_{epoch}.h5")

    def sample_interval(self, epoch):
        r, c = 5, 5
        z = np.random.normal(size=(25, self.latent_dim))
        gen_imgs = self.generator.predict(z)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap="gray")
                axs[i, j].axis("off")
                cnt += 1
        fig.savefig("./drive/MyDrive/images3/faces_%d.png" % epoch)
        plt.close()



In [1]:
bigan = BIGAN()

In [ ]:
# train model
bigan.train(epochs=30000, batch_size=50, sample_interval=500)

0 [D loss: 0.728278, acc: 50.00%] [G loss: 1.523060]
500 [D loss: 0.422459, acc: 50.00%] [G loss: 20.199303]
1000 [D loss: 0.854213, acc: 50.00%] [G loss: 4.517582]
1500 [D loss: 0.265337, acc: 50.00%] [G loss: 6.955372]
2000 [D loss: 0.105317, acc: 50.00%] [G loss: 11.199877]
2500 [D loss: 0.169401, acc: 50.00%] [G loss: 10.440197]
3000 [D loss: 0.492401, acc: 50.00%] [G loss: 3.929972]
3500 [D loss: 0.079565, acc: 50.00%] [G loss: 10.309497]
4000 [D loss: 0.806233, acc: 50.00%] [G loss: 21.193182]
4500 [D loss: 0.043731, acc: 50.00%] [G loss: 18.944431]
5000 [D loss: 0.143599, acc: 50.00%] [G loss: 14.566864]
5500 [D loss: 0.522592, acc: 50.00%] [G loss: 7.084386]
6000 [D loss: 0.139181, acc: 50.00%] [G loss: 12.827082]
6500 [D loss: 0.191748, acc: 50.00%] [G loss: 10.500836]
7000 [D loss: 0.090439, acc: 50.00%] [G loss: 15.176466]
7500 [D loss: 0.271819, acc: 50.00%] [G loss: 16.295193]
8000 [D loss: 0.169290, acc: 50.00%] [G loss: 24.564199]
8500 [D loss: 0.190759, acc: 50.00%] [G 